# 🎯 Multi-Touch Attribution Modeling

**Goal:** Implement and compare different attribution models to understand the true value of each advertising channel.

Attribution models covered:
- **Last-Touch Attribution** (baseline)
- **First-Touch Attribution**
- **Linear Attribution** (equal weight)
- **Position-Based Attribution** (40-20-40 model)
- **Time-Decay Attribution**

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

print("🎯 Attribution modeling environment ready!")

# Attribution model functions
class AttributionModels:
    """
    A class containing different attribution model implementations
    """
    
    @staticmethod
    def last_touch(journey_df):
        """Last-touch attribution - all credit to final touchpoint"""
        return journey_df.groupby('user_id').last()
    
    @staticmethod
    def first_touch(journey_df):
        """First-touch attribution - all credit to initial touchpoint"""
        return journey_df.groupby('user_id').first()
    
    @staticmethod
    def linear(journey_df):
        """Linear attribution - equal credit to all touchpoints"""
        attribution_df = journey_df.copy()
        attribution_df['attribution_weight'] = 1.0 / journey_df.groupby('user_id')['channel'].transform('count')
        return attribution_df
    
    @staticmethod
    def position_based(journey_df, first_weight=0.4, last_weight=0.4):
        """Position-based attribution (40-20-40 model)"""
        attribution_df = journey_df.copy()
        
        def calculate_position_weights(group):
            n = len(group)
            if n == 1:
                group['attribution_weight'] = 1.0
            elif n == 2:
                group.iloc[0, group.columns.get_loc('attribution_weight')] = first_weight + (1 - first_weight - last_weight) / 2
                group.iloc[1, group.columns.get_loc('attribution_weight')] = last_weight + (1 - first_weight - last_weight) / 2
            else:
                middle_weight = (1 - first_weight - last_weight) / (n - 2) if n > 2 else 0
                group.iloc[0, group.columns.get_loc('attribution_weight')] = first_weight
                group.iloc[-1, group.columns.get_loc('attribution_weight')] = last_weight
                for i in range(1, n-1):
                    group.iloc[i, group.columns.get_loc('attribution_weight')] = middle_weight
            return group
        
        attribution_df['attribution_weight'] = 0.0
        attribution_df = attribution_df.groupby('user_id').apply(calculate_position_weights)
        return attribution_df.reset_index(drop=True)
    
    @staticmethod
    def time_decay(journey_df, decay_rate=7):
        """Time-decay attribution - more recent touchpoints get more credit"""
        attribution_df = journey_df.copy()
        attribution_df['timestamp'] = pd.to_datetime(attribution_df['timestamp'])
        
        def calculate_time_weights(group):
            if len(group) == 1:
                group['attribution_weight'] = 1.0
                return group
            
            # Calculate days from conversion
            max_time = group['timestamp'].max()
            group['days_to_conversion'] = (max_time - group['timestamp']).dt.days
            
            # Apply exponential decay
            group['raw_weight'] = np.exp(-group['days_to_conversion'] / decay_rate)
            
            # Normalize weights to sum to 1
            total_weight = group['raw_weight'].sum()
            group['attribution_weight'] = group['raw_weight'] / total_weight
            
            return group
        
        attribution_df = attribution_df.groupby('user_id').apply(calculate_time_weights)
        return attribution_df.reset_index(drop=True)

print("Attribution model classes defined successfully!")